In [1]:
from scipy.io import arff
import pandas as pd
data = arff.loadarff('seismic-bumps.arff')
dataset = pd.DataFrame(data[0])
datainfo = pd.DataFrame(data[1])
print(dataset)
print(datainfo)


     seismic seismoacoustic shift   genergy   gpuls  gdenergy  gdpuls ghazard  \
0       b'a'           b'a'  b'N'   15180.0    48.0     -72.0   -72.0    b'a'   
1       b'a'           b'a'  b'N'   14720.0    33.0     -70.0   -79.0    b'a'   
2       b'a'           b'a'  b'N'    8050.0    30.0     -81.0   -78.0    b'a'   
3       b'a'           b'a'  b'N'   28820.0   171.0     -23.0    40.0    b'a'   
4       b'a'           b'a'  b'N'   12640.0    57.0     -63.0   -52.0    b'a'   
5       b'a'           b'a'  b'W'   63760.0   195.0     -73.0   -65.0    b'a'   
6       b'a'           b'a'  b'W'  207930.0   614.0      -6.0    18.0    b'a'   
7       b'a'           b'a'  b'N'   48990.0   194.0     -27.0    -3.0    b'a'   
8       b'a'           b'a'  b'N'  100190.0   303.0      54.0    52.0    b'a'   
9       b'a'           b'a'  b'W'  247620.0   675.0       4.0    25.0    b'a'   
10      b'a'           b'a'  b'N'   41950.0   135.0     -39.0   -36.0    b'a'   
11      b'a'           b'a' 

In [2]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy
# from sklearn.model_selection import train_test_split
# integer encode
label_encoder = LabelEncoder()
dataset.iloc[:,0] = label_encoder.fit_transform(dataset.iloc[:,0])
dataset.iloc[:,1] = label_encoder.fit_transform(dataset.iloc[:,1])
dataset.iloc[:,2] = label_encoder.fit_transform(dataset.iloc[:,2])
dataset.iloc[:,7] = label_encoder.fit_transform(dataset.iloc[:,7])
dataset.iloc[:,-1] = label_encoder.fit_transform(dataset.iloc[:,-1])

X = dataset.iloc[:,:-1]

y = dataset.iloc[:,-1]

X = X.values
y = y.values

# print(y)
print("dataset_size: ", y.size)
# X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

dataset_size:  2584


In [3]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='stratified',random_state=None,constant=None)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y, cv=10)

print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


[0.88030888 0.87258687 0.88803089 0.84169884 0.87209302 0.87209302
 0.8875969  0.89922481 0.8875969  0.84883721]
Accuracy: 0.88 (+/- 0.03)


In [4]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(clf, X, y, cv=10)

from sklearn.metrics import confusion_matrix
print("confusion_matrix: ",confusion_matrix(y,y_pred))

from sklearn.metrics import precision_score,recall_score
print("precision: ",precision_score(y,y_pred))
print("recall:", recall_score(y,y_pred))

confusion_matrix:  [[2268  146]
 [ 158   12]]
precision:  0.0759493670886076
recall: 0.07058823529411765


In [5]:
#oversampling
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler()
# X_over, y_over = ros.fit_resample(X, y)

from imblearn.over_sampling import SMOTE
ros = SMOTE()
X_over, y_over = ros.fit_resample(X, y)

print("dataset_size: ", y_over.size)

scores_over = cross_val_score(clf, X_over, y_over, cv=10)

print(scores_over)
print("Accuracy of Oversampling: %0.2f (+/- %0.2f)" % (scores_over.mean(), scores_over.std() * 2))

y_pred_over = cross_val_predict(clf, X_over, y_over, cv=10)

print("confusion matrix of oversampling: ",confusion_matrix(y_over,y_pred_over))

print("precision of oversampling: ",precision_score(y_over,y_pred_over))
print("recall of oversampling:", recall_score(y_over,y_pred_over))

Using TensorFlow backend.


dataset_size:  4828
[0.48553719 0.48347107 0.49173554 0.52479339 0.53112033 0.49792531
 0.50829876 0.52282158 0.48962656 0.48962656]
Accuracy of Oversampling: 0.50 (+/- 0.03)
confusion matrix of oversampling:  [[1218 1196]
 [1176 1238]]
precision of oversampling:  0.5086277732128184
recall of oversampling: 0.5128417564208783


In [6]:
#undersampling
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids()
X_under, y_under = cc.fit_resample(X, y)

print("dataset_size: ", y_under.size)

scores_under = cross_val_score(clf, X_under, y_under, cv=10)

print(scores_under)
print("Accuracy of Undersampling: %0.2f (+/- %0.2f)" % (scores_under.mean(), scores_under.std() * 2))

y_pred_under = cross_val_predict(clf, X_under, y_under, cv=10)

print("confusion matrix of undersampling: ",confusion_matrix(y_under,y_pred_under))

print("precision of undersampling: ",precision_score(y_under,y_pred_under))
print("recall of undersampling:", recall_score(y_under,y_pred_under))

dataset_size:  340
[0.5        0.44117647 0.52941176 0.41176471 0.44117647 0.35294118
 0.61764706 0.5        0.35294118 0.41176471]
Accuracy of Undersampling: 0.46 (+/- 0.16)
confusion matrix of undersampling:  [[78 92]
 [88 82]]
precision of undersampling:  0.47126436781609193
recall of undersampling: 0.4823529411764706


In [7]:
#combine oversampling and undersampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
X_combine, y_combine = smote_enn.fit_resample(X, y)

print("dataset_size: ", y_combine.size)

scores_combine = cross_val_score(clf, X_combine, y_combine, cv=10)

print(scores_combine)
print("Accuracy of combining oversampling and undersampling: %0.2f (+/- %0.2f)" % (scores_combine.mean(), scores_combine.std() * 2))

y_pred_combine = cross_val_predict(clf, X_combine, y_combine, cv=10)

print("confusion matrix of combining oversampling and undersampling: ",confusion_matrix(y_combine,y_pred_combine))

print("precision of combining oversampling and undersampling: ",precision_score(y_combine,y_pred_combine))
print("recall of combining oversampling and undersampling:", recall_score(y_combine,y_pred_combine))

dataset_size:  3172
[0.51410658 0.48264984 0.48895899 0.47318612 0.49842271 0.45741325
 0.51419558 0.53312303 0.55835962 0.4511041 ]
Accuracy of combining oversampling and undersampling: 0.50 (+/- 0.06)
confusion matrix of combining oversampling and undersampling:  [[651 770]
 [781 970]]
precision of combining oversampling and undersampling:  0.5574712643678161
recall of combining oversampling and undersampling: 0.5539691604797259


In [9]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf_fs = ExtraTreesClassifier(n_estimators=50)
clf_fs = clf_fs.fit(X_combine, y_combine)
model_fs = SelectFromModel(clf_fs, prefit=True)
X_fs = model_fs.transform(X_combine)
print("feature dimension:", X_fs.shape[1])             

y_fs = y_combine

scores_fs = cross_val_score(clf, X_fs, y_fs, cv=10)

print(scores_fs)
print("Accuracy of feature selection: %0.2f (+/- %0.2f)" % (scores_fs.mean(), scores_fs.std() * 2))

y_pred_fs = cross_val_predict(clf, X_fs, y_fs, cv=10)

print("confusion matrix of feature selection: ",confusion_matrix(y_fs,y_pred_fs))

print("precision of feature selection: ",precision_score(y_fs,y_pred_fs))
print("recall of feature selection:", recall_score(y_fs,y_pred_fs))

feature dimension: 8
[0.49216301 0.47318612 0.53943218 0.48580442 0.51419558 0.51419558
 0.45741325 0.5362776  0.49211356 0.57097792]
Accuracy of feature selection: 0.51 (+/- 0.07)
confusion matrix of feature selection:  [[649 772]
 [826 925]]
precision of feature selection:  0.5450795521508545
recall of feature selection: 0.528269560251285


In [10]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(C=0.5, penalty="l1", dual=False).fit(X_combine, y_combine)
model_fs_2 = SelectFromModel(lsvc, prefit=True)
X_fs_2 = model_fs_2.transform(X_combine)
print("feature dimension:", X_fs_2.shape[1])
y_fs_2 = y_combine

scores_fs_2 = cross_val_score(clf, X_fs_2, y_fs_2, cv=10)

print(scores_fs_2)
print("Accuracy of feature selection: %0.2f (+/- %0.2f)" % (scores_fs_2.mean(), scores_fs_2.std() * 2))

y_pred_fs_2 = cross_val_predict(clf, X_fs_2, y_fs_2, cv=10)

print("confusion matrix of feature selection: ",confusion_matrix(y_fs_2,y_pred_fs_2))

print("precision of feature selection: ",precision_score(y_fs_2,y_pred_fs_2))
print("recall of feature selection:", recall_score(y_fs_2,y_pred_fs_2))

feature dimension: 14
[0.50783699 0.50788644 0.51735016 0.4384858  0.49526814 0.55205047
 0.48895899 0.5362776  0.48264984 0.52365931]
Accuracy of feature selection: 0.51 (+/- 0.06)
confusion matrix of feature selection:  [[648 773]
 [792 959]]
precision of feature selection:  0.5536951501154734
recall of feature selection: 0.5476870359794404


//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
